In [1]:
import torch
from torch import nn, optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import functional as F
import matplotlib.pyplot as plt

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        # Max pool layer
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        # Convolutional layer 1
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        # Max pool layer
        out = self.pool(out)

        # Convolutional layer 2
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        # Max pool layer
        out = self.pool(out)

        # Flatten for fully connected layer
        out = out.view(out.size(0), -1)

        # Fully connected layer 1
        out = self.fc1(out)

        # Fully connected layer 2
        out = self.fc2(out)
        return out

#Data prep
# Define a transform to resize and normalize the images, and then convert them to PyTorch tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),  # Convert images to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values in the range [-1, 1]
])


In [2]:
# Define your parking spaces
parking_spaces = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']

# Path to your data
base_data_dir = '~/ParkEzAI/staff_tools/build_models/offline_data/training/'

# Dictionary to store the models
models_dict = {}

# Prepare the model for each parking space
for parking_space in parking_spaces:
    data_dir = base_data_dir + parking_space

    # Create ImageFolder dataset
    dataset = ImageFolder(data_dir, transform=transform)

    # Use all images for training
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Use the previously defined CNN class
    model = CNN()

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Number of epochs to train for
    num_epochs = 25

    # Move model to GPU if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # set the model to training mode
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            # Move data and labels to GPU if available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Clear the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss = train_loss / len(train_loader.dataset)

        print('Parking Space: {}, Epoch {}, Loss: {:.4f}'.format(parking_space, epoch + 1, train_loss))

    print('Finished Training for parking space ', parking_space)

    # save the model after training
    torch.save(model.state_dict(), f"{parking_space}_model.pth")

    # add the model to the dictionary
    models_dict[parking_space] = model


Parking Space: A1, Epoch 1, Loss: 11.9732
Parking Space: A1, Epoch 2, Loss: 2.8765
Parking Space: A1, Epoch 3, Loss: 0.0000
Parking Space: A1, Epoch 4, Loss: 0.0205
Parking Space: A1, Epoch 5, Loss: 0.0000
Parking Space: A1, Epoch 6, Loss: 0.0000
Parking Space: A1, Epoch 7, Loss: 0.0000
Parking Space: A1, Epoch 8, Loss: 0.0000
Parking Space: A1, Epoch 9, Loss: 0.0000
Parking Space: A1, Epoch 10, Loss: 0.0000
Parking Space: A1, Epoch 11, Loss: 0.0000
Parking Space: A1, Epoch 12, Loss: 0.0000
Parking Space: A1, Epoch 13, Loss: 0.0000
Parking Space: A1, Epoch 14, Loss: 0.0000
Parking Space: A1, Epoch 15, Loss: 0.0000
Parking Space: A1, Epoch 16, Loss: 0.0000
Parking Space: A1, Epoch 17, Loss: 0.0000
Parking Space: A1, Epoch 18, Loss: 0.0000
Parking Space: A1, Epoch 19, Loss: 0.0000
Parking Space: A1, Epoch 20, Loss: 0.0000
Parking Space: A1, Epoch 21, Loss: 0.0000
Parking Space: A1, Epoch 22, Loss: 0.0000
Parking Space: A1, Epoch 23, Loss: 0.0000
Parking Space: A1, Epoch 24, Loss: 0.0000


Parking Space: B5, Epoch 16, Loss: 0.0000
Parking Space: B5, Epoch 17, Loss: 0.0000
Parking Space: B5, Epoch 18, Loss: 0.0000
Parking Space: B5, Epoch 19, Loss: 0.0000
Parking Space: B5, Epoch 20, Loss: 0.0000
Parking Space: B5, Epoch 21, Loss: 0.0000
Parking Space: B5, Epoch 22, Loss: 0.0000
Parking Space: B5, Epoch 23, Loss: 0.0000
Parking Space: B5, Epoch 24, Loss: 0.0000
Parking Space: B5, Epoch 25, Loss: 0.0000
Finished Training for parking space  B5
Parking Space: B6, Epoch 1, Loss: 9.3331
Parking Space: B6, Epoch 2, Loss: 4.4090
Parking Space: B6, Epoch 3, Loss: 0.0649
Parking Space: B6, Epoch 4, Loss: 1.8433
Parking Space: B6, Epoch 5, Loss: 0.0000
Parking Space: B6, Epoch 6, Loss: 0.0000
Parking Space: B6, Epoch 7, Loss: 0.0000
Parking Space: B6, Epoch 8, Loss: 0.0000
Parking Space: B6, Epoch 9, Loss: 0.0000
Parking Space: B6, Epoch 10, Loss: 0.0000
Parking Space: B6, Epoch 11, Loss: 0.0000
Parking Space: B6, Epoch 12, Loss: 0.0000
Parking Space: B6, Epoch 13, Loss: 0.0000
Par

In [4]:
from sklearn.metrics import precision_recall_fscore_support

# Initialize counters
total_correct = 0
total_images = 0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

# Initialize lists to hold model outputs and labels
all_labels = []
all_predictions = []

# Path to your data
test_data_dir = '~/ParkEzAI/staff_tools/build_models/offline_data/testing/'


# For each model in the models dictionary
for parking_space, model in models_dict.items():
    # Create a path to the testing data for this parking space
    test_dir = test_data_dir + parking_space

    # Create ImageFolder dataset for testing data
    test_dataset = ImageFolder(test_dir, transform=transform)

    # Create a DataLoader for the testing data
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Switch the model to evaluation mode
    model.eval()

    correct = 0
    total = 0

    # No gradient computation during evaluation
    with torch.no_grad():
        for images, labels in test_loader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Convert the labels and predictions to lists and add them to our tracking lists
            all_labels.extend(labels.tolist())
            all_predictions.extend(predicted.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    total_correct += correct
    total_images += total

    print(f'Test accuracy for {parking_space}: {100 * correct / total}%')
    print(f'Correct / Total for {parking_space}: {correct} / {total}')

print(f'Overall Test accuracy: {100 * total_correct / total_images}%')
print(f'Overall Correct / Total: {total_correct} / {total_images}')

# Calculate precision, recall, f-score and support
precision, recall, f1_score, support = precision_recall_fscore_support(all_labels, all_predictions)

# Print per-class metrics
for i in range(2):
    if class_total[i] > 0:
        accuracy = round((100 * class_correct[i] / class_total[i]), 4)
        precision_score = round((precision[i] * 100), 4)
        recall_val = round((recall[i] * 100), 4)
        f1_val = round((f1_score[i] * 100), 4)
        
        print(f'Accuracy of {i} : {accuracy}%')
        print(f'Precision of {i} : {precision_score}%')
        print(f'Recall of {i} : {recall_val}%')
        print(f'F1 score of {i} : {f1_val}%')
    else:
        print(f'Accuracy of {i} : N/A (no instances)')



Test accuracy for A1: 100.0%
Correct / Total for A1: 39 / 39
Test accuracy for A2: 100.0%
Correct / Total for A2: 39 / 39
Test accuracy for A3: 100.0%
Correct / Total for A3: 39 / 39
Test accuracy for B1: 100.0%
Correct / Total for B1: 39 / 39
Test accuracy for B2: 100.0%
Correct / Total for B2: 39 / 39
Test accuracy for B3: 97.43589743589743%
Correct / Total for B3: 38 / 39
Test accuracy for B4: 100.0%
Correct / Total for B4: 39 / 39
Test accuracy for B5: 100.0%
Correct / Total for B5: 39 / 39
Test accuracy for B6: 100.0%
Correct / Total for B6: 39 / 39
Overall Test accuracy: 99.71509971509971%
Overall Correct / Total: 350 / 351
Accuracy of 0 : 100.0%
Precision of 0 : 99.2537%
Recall of 0 : 100.0%
F1 score of 0 : 99.6255%
Accuracy of 1 : 99.5413%
Precision of 1 : 100.0%
Recall of 1 : 99.5413%
F1 score of 1 : 99.7701%
